In [91]:
from common import *

In [145]:
logavg = lambda x,y: np.expm1((np.log1p(x)+np.log1p(y))/2)
keepSigFig = lambda n: lambda x: round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1)) if x else x

In [194]:
class LSD(O()):
    class discrete(O()):
        raw = {
            ('max_depth','num_leaves'): #10
                [(6,1<<6),(9,1<<7),(9,1<<9),(12,1<<8),(12,1<<10),(12,1<<12),(-1,1<<8),(-1,1<<10),(-1,1<<12),(-1,1<<14)],
            #('max','num'): #10
            #    [(6,6),(9,7),(9,9),(12,8),(12,10),(12,12),(-1,8),(-1,10),(-1,12),(-1,14)],
        }
        
    class oneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 2
        data = {
            'min_data_in_leaf': O(a=[1,60,375], cast=round),
            'min_sum_hessian_in_leaf': O(a=[0,50,200]),
            'lambda_l1': O(a=[0,.02,.2], b=0, lim=1),
            'lambda_l2': O(a=[0,.02,.2], b=0, lim=1),
        }

class LightSearch():
    mix = logavg
    
    def __init__(self, lsd):
        self.lsd = O.mycopy(lsd)
        self.setup_data()
        
    def setup_data(self):
        dsc = self.lsd.discrete
        dsc.keys = list(flatten( dsc.raw.keys() ))
        dsc.assigns = [list(flatten(x)) for x in product(* dsc.raw.values() )]
        obo = self.lsd.oneByOne
        for k,v in dict.items(obo.data):
            for i,x in dict.items(obo.default):
                if i not in v:
                    v[i] = x
    
    def search(self):
        dsc, obo = self.lsd.discrete, self.lsd.oneByOne
        for assign in dsc.assigns:
            params = dict(zip(dsc.keys, assign))
            pass
        
    def one_by_one(self):
        obod = self.lsd.oneByOne.data
        ranges = {k: v.a for k,v in dict.items(obod)}
        params = {k: (v.b if b in v else v.a[1]) for k,v in dict.items(obod)}
        scores = {k: [-np.inf, -np.inf] for k in params}
        isdone = {k: False for k in params}
        prev_base_score = -np.inf
        for i in range(9999999999):
            base_score = yield params
            if base_score <= prev_base_score:
                break
            isdone = {k: v or i>=obod[key].lim for k,v in dict.items(params)}
            if all(isdone.values()):
                break
            for key in list(params):
                if i >= obod[key].lim:
                    continue
                orig = params[key]
                params[key] = obod[key].cast(self.mix(ranges[key][0], ranges[key][1]))
                scores[key][0] = yield params
                params[key] = obod[key].cast(self.mix(ranges[key][1], ranges[key][2]))
                scores[key][1] = yield params
                params[key] = orig
            prev_base_score = base_score
            

In [193]:
[list(flatten(x)) for x in product(*LSD.fixed.values())]

[[6, 64],
 [9, 128],
 [9, 512],
 [12, 256],
 [12, 1024],
 [12, 4096],
 [-1, 256],
 [-1, 1024],
 [-1, 4096],
 [-1, 16384]]